# Data Discovery: CMR-STAC API  

## Timing  

- Exercise: 30 min  

## Summary  

In this example we will access the NASA's Harmonized Landsat Sentinel-2 (HLS) version 2 assets, which are archived in cloud optimized geoTIFF (COG) format in the LP DAAC Cumulus cloud space. The COGs can be used like any other geoTIFF file, but have some added features that make them more efficient within the cloud data access paradigm. These features include: overviews and internal tiling. Below we will demonstrate how to leverage these features.  

### But first, what is STAC?  

[SpatioTemporal Asset Catalog (STAC)](https://stacspec.org/) is a specification that provides a common language for interpreting geospatial information in order to standardize indexing and discovering data.  

The [STAC specification](https://stacspec.org/core.html) is made up of a collection of related, yet independent specifications that when used together provide search and discovery capabilities for remove assets.  

#### Four STAC Specifications  
                                                                                                                  
[STAC Item](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md) (aka Granule)  
[STAC Catalog](https://github.com/radiantearth/stac-spec/blob/master/catalog-spec/catalog-spec.md) (aka DAAC Archive)  
[STAC Collection](https://github.com/radiantearth/stac-spec/blob/master/collection-spec/collection-spec.md) (aka Data Product)  
[STAC API](https://github.com/radiantearth/stac-api-spec)  

In the following sections, we will explore each of STAC element using NASA's Common Metadata Repository (CMR) STAC application programming interface (API), or [CMR-STAC API](https://github.com/nasa/cmr-stac) for short.  

### CMR-STAC API  

The [CMR-STAC](https://github.com/nasa/cmr-stac) API is NASA's implementation of the STAC API specification for all NASA data holdings within EOSDIS. The current implementation does not allow for querries accross the entire NASA catalog. Users must execute searches within provider catalogs (e.g., LPCLOUD) to find the STAC Items they are searching for. All the providers can be found at the CMR-STAC endpoint here: <https://cmr.earthdata.nasa.gov/stac/>.  

In this exercise, we will query the **LPCLOUD** provider to identify STAC Items from the Harmonized Landsat Sentinel-2 (HLS) collection that fall within our region of interest (ROI) and within our specified time range.  

## What you will learn from this tutorial  

- how to connect to NASA CMR-STAC API using Python's `pystac-client`  
- how to navigate CMR-STAC records  
- how to read in a geojson file using `geopandas` to specify your region of interest
- how to use the CMR-STAC API to search for data 
- how to perform post-search filtering of CMR-STAC API search result in Python  
- how to extract and save data access URLs for geospatial assets 

---

## Import Required Packages

In [ ]:
#

## Explored available NASA Providers  

In [ ]:
#

### Connect to the CMR-STAC API

In [ ]:
#

We'll create a `providers` variable so we can take a deeper look into available data providers - subcategories are referred to as "children". We can then print them as a for loop.

In [ ]:
#

## Connect to the `LPCLOUD` Provider/STAC Catalog

For this next step we need the `provider title` (e.g., LPCLOUD) from above. We will add the provider to the end of the CMR-STAC API URL (i.e., `https://cmr.earthdata.nasa.gov/stac/`) to connect to the LPCLOUD STAC Catalog.

In [ ]:
#

## List STAC Collections

We'll create a `products` variable to view deeper in the STAC Catalog. 

In [ ]:
#

### Print one of the STAC Collection records

To view the `products` variable we just created, let's look at one entry as a dictionary. 

In [ ]:
#

### Print the STAC Collection `id`s with their `title`

In the above output, `id` and `title` are two elements of interest that we can print for all `products` using a for loop. 

TODO: pystac/pystac-client objects

In [ ]:
#

## Search for Granules/STAC Items - Set up query parameters to submit to the CMR-STAC API

We will define our region of interest (ROI) using the geojson file from the previous exercise, while also specifying the data collections and time range of needed for our example.

### Read in a geojson file

Reading in a geojson file with geopandas will return the geometry of our polygon (our ROI).

In [ ]:
#

### Visualize contents of geojson file

We can use that geometry to visualize the polygon: here, a square. But wait for it –

In [ ]:
#

We can plot the polygon using the `geoviews` package that we imported as `gv` with 'bokeh' and 'matplotlib' extensions. The following has reasonable width, height, color, and line widths to view our polygon when it is overlayed on a base tile map. 

In [ ]:
#

We will now start to specify the search criteria we are interested in, i.e, the **date range**, the **region of interest** (ROI), and the **data collections**, that we will pass to the STAC API.

### Extract the coordinates for the region of interest (ROI)

In [ ]:
#

So, what just happen there? Let's take a quick detour to break it down.

|   |   |
|---|---|
|field.to_json() | geojson representation as a string|  
|json.loads()    | parse a json string into a Python Dictionary|
|features         | Dictionary key that contrains the geometry object with coordinates (returned as a list)|
|geometry        | Dictionary key that contains the coordinates for the ROI|

### Specify date range

Next up is to specify our date range using [ISO_8601 date formatting](https://en.wikipedia.org/wiki/ISO_8601). 

In [ ]:
#

### Specify the STAC Collections

STAC Collection is synonomous with what we usually consider a NASA data product. Desired STAC Collections are submitted to the search API as a list containing the collection `id`. We can use the `id`s that we printed from our `products` for loop above. Let's focus on S30 and L30 collections. 

In [ ]:
#

## Search the CMR-STAC API with our search criteria

Now we can put all our search criteria together using `catalog.search` from the `pystac_client` package.

In [ ]:
#

### Print out how many STAC Items match our search query

In [ ]:
#

We now have a search object containing the STAC Items that matched our query. Now, let's pull out all of the STAC Items (as a PySTAC ItemCollection object) and explore the contents (i.e., the STAC Items)

In [ ]:
#

Let's list a few of these `item_collection`s:

In [ ]:
#

### Print an item as a dictionary

We can view a single item as a dictionary, as we did above with `products`.

In [ ]:
#

TODO Aaron: identify a few things of note 

## Filtering STAC Items

While the CMR-STAC API is a powerful search and discovery utility, it is still maturing and currently does not have the full gamut of filtering capabilities that the STAC API specification allows for. Hence, additional filtering is required if we want to filter by a property, for example cloud cover. Below we will loop through and filter the item_collection by a specified cloud cover as well as extract the band we'd need to do an Enhanced Vegetation Index (EVI) calculation for a future analysis.

We'll make a `cloudcover` variable where we will set the maximum allowable cloud cover and extract the band links for those Items that match or are less than the max cloud cover.

In [ ]:
#

We will also specify the STAC Assets (i.e., bands/layers) of interest for both the S30 and L30 collections (also in our `collections` variable above).

*TODO Aaron*. We'll set these bands that are reasonable for EVI calculations: read more about B8A and friends here: TODO.

In [ ]:
#

And now to loop through and filter the `item_collection` by cloud cover and bands:

In [ ]:
#

The filtering done in the previous steps produces a list of links to STAC Assets. Let's print out the first ten links.

In [ ]:
#

**NOTICE** that in the list of links that we have multiple tiles, i.e. **T14TKL** & **T13TGF**, that intersect with our region of interest. TODO: We can notice this by HOW. 

These two tiles represent neighboring Universal Transverse Mercator (UTM) zones. We will split the list of links into separate lists for each tile. 

We now have a list of links to data assets that meet our search and filtering criteria. The commands that follow will split this list into logical groupings using python routines.

## Split Data Links List into Logical Groupings

TODO Aaron: a bit more description about what all these steps do :)

Split by UTM tile specified in the file name (e.g., T14TKL & T13TGF)

In [ ]:
#

In [ ]:
#

### Print dictionary keys and values, i.e. the data links

In [ ]:
#

In [ ]:
#

Now we will create a separate list of data links for each tile

In [ ]:
#

### Print band/layer links for HLS tile T13TGF

In [ ]:
#

### Split the links by band

In [ ]:
#

In [ ]:
#

In [ ]:
#

In [ ]:
#

## Save links to a text file

To complete this exercise, we will save the individual link lists as separate text files with descriptive names.

### Write links from CMR-STAC API to a file

In [ ]:
#

### Write links to file for S3 access

In [ ]:
#